# Le partitionnement des données

Le partitionnement (*clustering*) est une technique courante des statistiques multivariées pour effectuer des tâches de regroupement entre variables afin de révéler une structure sous-jacente. Il s’agit d’une méthode exploratoire qui aide à la classification des données en regroupant les individus dans des ensembles cohérents où la variance intra-groupes est minimisée quand la variance inter-groupes est, elle, maximisée.

Importons les modules qui seront nécessaires :

In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.cluster import hierarchy
from scipy.cluster.vq import kmeans, vq
from scipy.spatial.distance import pdist, squareform

## Une matrice de dissimilarité

### Avec des variables catégorielles

Prenons les réponses de cinq étudiant·es à un test comportant dix questions :

In [ ]:
n_students = 5
n_questions = 10

data = [
    [
        random.choice(['A', 'B', 'C', 'D'])
        for _ in range(n_questions)
    ]
    for _ in range(n_students)
]

df = pd.DataFrame(data, index=[f"Student {i+1}" for i in range(n_students)], columns=[f'Q{i+1}' for i in range(0, n_questions)])

display(df)

Et faisons la comparaison deux à deux pour comptabiliser le nombre de fois où leurs réponses divergent. Enfin, normalisons en divisant le résultat par le nombre de questions afin d’obtenir un score entre 0 et 1 où 0 correspond à des étudiant·es aux réponses similaires et 1 des étudiant·es qui n’auront jamais répondu pareil :

In [ ]:
data_array = df.values

# calculate differences between rows
row_diffs = (data_array[:, None] != data_array).sum(axis=2)

# normalize
dissimilarity_matrix = row_diffs / n_questions

# matrix to a DataFrame
dissimilarity_df = pd.DataFrame(dissimilarity_matrix, index=df.index, columns=df.index)

display(dissimilarity_df)

Il est à présent facile d’identifier les paires d’étudiant·es dont les réponses se ressemblent le plus :

In [ ]:
# minimum score > 0 to exclude pairs consisting of the same student
min_score = dissimilarity_df[dissimilarity_df > 0].min().min()

# all the pairs concerned
clusters = dissimilarity_df[dissimilarity_df == min_score].stack().index.tolist()

display(clusters)

### Avec des variables numériques

Dans l’exemple précédent, les variables enregistraient des données catégorielles. Si maintenant nous prenons l’exemple d’une dizaine de textes avec des scores sur 20 dans cinq catégories :

In [ ]:
# 10 texts with a score on 5 categories
n_texts = 10
categories = ["Sciences", "Politique", "Littérature", "Journalisme", "Philosophie"]

df = pd.DataFrame(
    data=np.random.randint(0, 21, size=(n_texts, len(categories))),
    index=[f"Text {i + 1}" for i in range(0, n_texts)],
    columns=categories
)

display(df)

Il n’est plus question ici de repérer les catégories où les textes ont obtenu des scores différents, aussi la première étape consiste à calculer une matrice de corrélation :

In [ ]:
correlation_matrix = df.corr()

display(correlation_matrix)

Cette matrice ressort des coefficients variant de -1 à 1 pour exprimer la corrélation entre chaque paire de variables. Pour la transformer en une matrice de dissimilarité, il suffit de calculer l’inverse de la corrélation :

In [ ]:
dissimilarity_matrix = 1 / correlation_matrix

display(dissimilarity_matrix)

Une formule alternative consiste à calculer plutôt l’opposé de la corrélation :

In [ ]:
dissimilarity_matrix = 1 - correlation_matrix

display(dissimilarity_matrix)

Puis à normaliser afin d’obtenir des scores dans l’intervalle $[0,1]$ :

In [ ]:
display(dissimilarity_matrix / 2)

De là, nous pouvons effectuer des prédictions sur les *clusters* formés entre les catégories. Peut-être la littérature et la philosophie sont-elles liées par leur coefficient de dissimilarité et, comme la philosophie et la politique sont elles-mêmes reliées, pourrions-nous en conclure que les trois disciplines forment un groupe.

**Remarque :** les données sont générées aléatoirement aussi les groupes seront-ils toujours différents.

## Une matrice de distances

Bien souvent, lorsque l’on calcule la dissimilarité entre des vecteurs, on calcule la distance euclidienne :

$$
d(\mathbf{x}, \mathbf{y}) = \sqrt{\sum_{i=1}^{n} (x_i - y_i)^2}
$$

### Calcul de la distance euclidienne

La fonction peut s’interpréter en Python :

In [ ]:
def euclidean_distance(*, a:list, b:list) -> float:
    """Euclidean distance between two vectors.
    
    Keyword arguments:
    a -- first vector
    b -- second vector
    """
    # difference between indices
    coords = [
        (x - y) ** 2
        for x, y in zip(a, b)
    ]
    # distance = square root of the sum of coords
    return sum(coords) ** .5

Pour l’appliquer à notre jeu de données, on va d’abord générer une matrice de distances nulle avant de la remplir en se servant de la symétrie :

In [ ]:
# a null matrix
pairwise_distances = np.zeros((n_texts, n_texts))

for i in range(n_texts):
    for j in range(i, n_texts):  # start at 'i' to avoid calculating b-a pair if a-b already stored
        dist = euclidean_distance(a=df.iloc[i].values.tolist(), b=df.iloc[j].values.tolist())
        pairwise_distances[i, j] = dist
        pairwise_distances[j, i] = dist  # matrix is symetric

display(pairwise_distances)

### Calcul avec *Numpy*

Calculer la distance euclidienne entre deux vecteurs *a* et *b* revient à calculer la norme du vecteur différence $a - b$.

$$
\|\mathbf{v}\| = \sqrt{\sum_{i=1}^{n} v_i^2}
$$

Or, la fonction `.linalg.norm()` de *Numpy* permet d’obtenir directement la norme d’un vecteur ; aussi pouvons-nous nous passer de la fonction `euclidean_distance()` définie plus haut :

In [ ]:
# a null matrix
pairwise_distances = np.zeros((n_texts, n_texts))

for i in range(n_texts):
    # find vector a
    vector_a = df.iloc[i].values
    for j in range(i, n_texts):
        # find vector b
        vector_b = df.iloc[j].values
        dist = np.linalg.norm(vector_a - vector_b)
        pairwise_distances[i, j] = dist
        pairwise_distances[j, i] = dist

display(pairwise_distances)

### Une bibliothèque scientifique spécialisée

La bilbiothèque *Scipy* fournit des méthodes spécialisées pour le calcul scientifique. Utilisons les fonctions `pdist` et `squareform` du module `spatial.distance` :

In [ ]:
# euclidean matrix
distances = pdist(df.values, metric='euclidean')

# to a square matrix
distance_matrix = squareform(distances)

# to a dataframe
distance_df = pd.DataFrame(distance_matrix, index=df.index, columns=df.index)

display(distance_df)

## Le regroupement hiérarchique ascendant

Jusqu’ici, les techniques de partitionnement proposées étaient plutôt simples et élémentaires : elles permettaient de dégager des appariements, tout au plus des regroupements de trois éléments. L’objectif à présent est de présenter une méthode plus systématique qui va, à chaque étape, effectuer des regroupements deux à deux jusqu’à ce qu’il ne reste plus qu’un seul groupe constitué de l’ensemble des autres.

On peut matérialiser cette méthode de classification ascendante hiérarchique (HAC pour *Hierarchical Agglomerative Classification*) grâce à un dendogramme :

In [ ]:
# by default, euclidean distance
Z = hierarchy.linkage(df, method='single')

# plot dendrogram
plt.figure(figsize=(15, 7))
plt.title('Dendrogram')
plt.xlabel('Samples')
plt.ylabel('Distance')
_ = hierarchy.dendrogram(
    Z,
    labels=df.index,
    leaf_rotation=90,
    leaf_font_size=10,
    color_threshold=0.7 * max(Z[:, 2])
)
plt.show()

### Le clustering par liaison simple

![Single-linkage clustering](./figs/single-linkage_clustering.svg)

Une technique assez intuitive de regroupement repose sur l’idée que clusters sont reliés si deux de leurs composants sont plus proches l’un de l’autre que de n’importe quel autre composant d’un autre cluster. Pour illustrer cela, prenons quatre clusters *A*, *B*, *C* et *D* :

|       | A      | B      | C   | D |
|-------|--------|--------|----|----|
| **A** | 0      | **12** | 13 | 34 |
| **B** | **12** | 0      | 16 | 28 |
| **C** | 13     | 16     | 0  | 15 |
| **D** | 34     | 28     | 15 | 0  |

Ici, les clusters *A* et *B* sont plus proches l’un de l’autre que de n’importe quel autre cluster, aussi pouvons-nous les regrouper. L’étape suivante va demander de réactualiser le tableau des distances où l’on remarque que les clusters $(A,B)$ et *C* sont désormais les plus proches :

|            | (A,B)  | C      | D  |
|------------|--------|--------|----|
| **(A,B)**  | 0      | **13** | 28 |
| **C**      | **13** | 0      | 15 |
| **D**      | 28     | 15     | 0  |

Après actualisation du tableau, nous obtenons :

|               | ((A,B),C) | D  |
|---------------|-----------|----|
| **((A,B),C)** | 0         | 15 |
| **D**         | 15        | 0  |

À la fin, le cluster unique que nous avons formé est : $(((A,B),C), D)$.

#### Décomposition étape par étape

L’approche du clustering par liaison simple (ou *single-linkage clustering* en anglais) est dite ascendante (ou *bottom-up*) dans le sens où elle forme des clusters élément par élément.

À chaque étape, il est déjà possible de calculer le point de jonction entre les clusters, logiquement situé à équidistance entre leurs composants les plus proches. Par exemple, à la première étape, les clusters *A* et *B* sont à une distance de 12 l’un de l’autre. Si nous traçons une droite entre les deux et que nous devions en trouver le milieu, nous diviserions 12 par deux pour en déduire que le point de jonction entre les deux clusters se trouve à 6. À la seconde étape, le point de jonction entre les clusters *(A,B)* et *C* est situé à $13 \div 2 = 6.5$.

**Remarque :** on appellera plutôt ce point de jonction un nœud.

Ensuite, la partie cruciale consiste à réactualiser la matrice des distances selon la fonction de liaison exprimée par la formule :

$$
D(X, Y) = \min_{x \in X, y \in Y} d(x, y)
$$

Où :

- *X* et *Y* sont des clusters ;
- *x* et *y* des composants de ces clusters.

En reprenant notre exemple, et après avoir identifié que *A* et *B* étaient les clusters les plus proches et les avoir regroupés, nous devons déterminer lequel des deux était le plus proche de *C* et de *D* :

- la distance *AC* est de 13, quand *BC* est de 16, aussi considère-t-on que le cluster $(A,B)$ se situe à une distance de 13 de *C* ;
- $D((A,B),D) = \min{(D(A,D), D(B,D))} = \min{(34,28)} = 28$

#### Algorithme de résolution

Si l’on devait concevoir à la main un programme qui effectue un clustering par liaison simple, nous répertorerions les actions suivantes à mener :

- calculer la distance entre les clusters ;
- trouver les points les plus proches entre tous les clusters ;
- déterminer entre tous les clusters qui sont les plus proches.

Définissons les fonctions nécessaires :

In [ ]:
def single_linkage(k:list, l:list, m) -> float:
    """Calculate the minimum distance between clusters,
    according to the single-linkage method:
    d(k, l) = min d(u, v)

    Args:
        k -- a cluster (collection of points)
        l -- another cluster
        m -- matrix

    Usage example:
        min_distance = single_linkage([0,3], [6,2,9], m)
    """
    return min(np.linalg.norm(m[p_k] - m[p_l]) for p_k in k for p_l in l)

def points_in_cluster(cluster):
    """A cluster is a collection of points."""
    return [int(point) for point in cluster.split(',')]

def dist_between_clusters(n, clusters, m):
    return [
        0 if n == cluster else single_linkage(points_in_cluster(n), points_in_cluster(cluster), m)
        for cluster in clusters.loc[n].index
    ]

def nearest_clusters(m):
    """Find the nearest clusters in a distance matrix."""

    # looking for the minimal distance
    min_dist = m[m > 0].min().min()

    # which clusters are concerned?
    c_1, c_2 = np.where(m == min_dist)[0]

    # and their names?
    c_1_name = m.iloc[c_1].name
    c_2_name = m.iloc[c_2].name

    return (c_1_name, c_2_name)

**1e étape :** calculer une matrice des distances où au départ chaque point est un cluster.

In [ ]:
# a copy of the distance matrix
cluster_df = pd.DataFrame(distance_df.values, index=[str(i) for i in range(len(distance_df))], columns=[str(i) for i in range(len(distance_df))])

# how many clusters at the end?
n = 2

# how many steps in total?
n_steps = len(cluster_df) - n

**2e étape :** trouver les clusters les plus proches et les regrouper (à répéter autant de fois que nécessaire).

In [ ]:
for i in range(n_steps):

    # find the nearest clusters
    c_1, c_2 = nearest_clusters(cluster_df)
    new_cluster = ','.join([c_1, c_2])

    # new cluster in town
    cluster_df.loc[new_cluster] = np.zeros(len(cluster_df))
    cluster_df[new_cluster] = np.zeros(len(cluster_df))

    # update distance matrix
    cluster_df.loc[new_cluster] = dist_between_clusters(new_cluster, cluster_df, distance_df.values)
    cluster_df[new_cluster] = dist_between_clusters(new_cluster, cluster_df, distance_df.values)

    # delete merged clusters
    cluster_df.drop([c_1, c_2], axis=0, inplace=True)
    cluster_df.drop([c_1, c_2], axis=1, inplace=True)

    # result
    print(f"Step {i + 1}: {','.join(str(int(x) + 1) for x in new_cluster.split(','))}")

### Le clustering par liaison complète

![Complete-linkage clustering](./figs/complete-linkage_clustering.svg)

Autre technique de partitionnement hiérarchique, le clustering par liaison complète (ou *complete-linkage clustering* en anglais) ne va pas considérer, contrairement au clustering par liaison simple, que la distance des points les plus proches de deux clusters définisse la distance entre les clusters, mais que ce soit plutôt la distance des points les plus éloignés.

La fonction de laison s’exprime désormais par l’expression mathématique :

$$
D(X, Y) = \max_{x \in X, y \in Y} d(x, y)
$$

En reprenant l’exemple plus haut, à la 1e étape les clusters *A* et *B* sont toujours les plus proches :

|       | A      | B      | C   | D |
|-------|--------|--------|----|----|
| **A** | 0      | **12** | 13 | 34 |
| **B** | **12** | 0      | 16 | 28 |
| **C** | 13     | 16     | 0  | 15 |
| **D** | 34     | 28     | 15 | 0  |

Après réactualisation du tableau des distances avec la fonction de laison complète, nous obtenons :

|            | (A,B) | C      | D      |
|------------|-------|--------|--------|
| **(A,B)**  | 0     | 16     | 34     |
| **C**      | 16    | 0      | **15** |
| **D**      | 34    | **15** | 0      |

Avec cette technique, le cluster $(A,B)$ s’est éloigné de *C* et de *D*, si bien que ces deux derniers sont désormais les plus proches l’un de l’autre :

|               | (A,B) | (C,D) |
|---------------|-------|-------|
| **(A,B)**     | 0     | 34    |
| **(C,D)**     | 34    | 0     |

À la fin, le cluster unique que nous avons formé est : $((A,B), (C,D))$.

#### Décomposition par étapes

À chaque étape, il convient de regrouper d’abord deux clusters en fonction de leur proximité puis de recalculer la distance qui les sépare des autres en prenant le point le plus éloigné.

Ainsi, après avoir identifié dans notre exemple que *A* et *B* étaient les clusters les plus proches et les avoir regroupés, nous devons déterminer lequel des deux était le plus **éloigné** de *C* et de *D* :

- la distance *AC* est de 13, quand *BC* est de 16, aussi considère-t-on que le cluster $(A,B)$ se situe à une distance de 16 de *C* ;
- $D((A,B),D) = \max{(D(A,D), D(B,D))} = \max{(34,28)} = 34$

#### Algorithme de résolution

L’algorithme est pour ainsi dire identique à celui du clustering par liaison simple. Nous avons juste besoin d’une fonction spécifique :

In [ ]:
def complete_linkage(k:list, l:list, m) -> float:
    """Calculate the maximum distance between clusters,
    according to the complete-linkage method:
    d(k, l) = max d(u, v)

    Args:
        k -- a cluster (collection of points)
        l -- another cluster
        m -- matrix

    Usage example:
        max_distance = complete_linkage([0,3], [6,2,9], m)
    """
    return max(np.linalg.norm(m[p_k] - m[p_l]) for p_k in k for p_l in l)

Et de redéfinir `dist_between_clusters()` en faisant appel à la fonction `complete_linkage()` :

In [ ]:
def dist_between_clusters(n, clusters, m):
    return [
        0 if n == cluster else complete_linkage(points_in_cluster(n), points_in_cluster(cluster), m)
        for cluster in clusters.loc[n].index
    ]

Pour le reste, la procédure est similaire :

In [ ]:
# a copy of the distance matrix
cluster_df = pd.DataFrame(distance_df.values, index=[str(i) for i in range(len(distance_df))], columns=[str(i) for i in range(len(distance_df))])

# how many clusters at the end?
n = 3

# how many steps in total?
n_steps = len(cluster_df) - n

In [ ]:
for i in range(n_steps):

    # find the nearest clusters
    c_1, c_2 = nearest_clusters(cluster_df)
    new_cluster = ','.join([c_1, c_2])

    # new cluster in town
    cluster_df.loc[new_cluster] = np.zeros(len(cluster_df))
    cluster_df[new_cluster] = np.zeros(len(cluster_df))

    # update distance matrix
    cluster_df.loc[new_cluster] = dist_between_clusters(new_cluster, cluster_df, distance_df.values)
    cluster_df[new_cluster] = dist_between_clusters(new_cluster, cluster_df, distance_df.values)

    # delete merged clusters
    cluster_df.drop([c_1, c_2], axis=0, inplace=True)
    cluster_df.drop([c_1, c_2], axis=1, inplace=True)

    # result
    print(f"Step {i + 1}: {','.join(str(int(x) + 1) for x in new_cluster.split(','))}")

### Le clustering par liaison centroïde

![Clustering par liaison centroïde](./figs/centroid-linkage_clustering.svg)

Dans cette relation, la distance entre deux clusters est la norme au carré du vecteur différence de leurs centroïdes, un centroïde étant simplement la moyenne d’une coordonnée :

$$
D(X, Y) = \| \mu_X - \mu_Y \|^2
$$

Cette méthode est un peu plus délicate que les précédentes dans la mesure où elle considère la distance entre des centroïdes et pas seulement entre des points.

**Remarque :** afin de marquer plus nettement la séparation entre les clusters, la formule de la distance euclidienne est élevée au carré.

Considérons cinq points dans un espace en deux dimensions et leur matrice des distances, au carré cette fois-ci :

In [ ]:
points = np.random.rand(5, 2) * 10
distances = pdist(points) ** 2
distance_matrix = squareform(distances)

Trouvons d’abord la plus petite distance et révélons l’indice des clusters concernés :

In [ ]:
# find the minimal distance
min_dist = distance_matrix[distance_matrix > 0].min().min()

# which clusters are concerned?
c_1, c_2 = np.where(distance_matrix == min_dist)[0]

Maintenant que les clusters `c_1` et `c_2` ont été identifiés comme les plus proches, il nous faut calculer leur centroïde :

In [ ]:
centroid = (points[c_1] + points[c_2]) / 2

Ce centroïde devient un nouveau point dans l’espace en remplacement des points derrière `c_1` et `c_2` :

In [ ]:
points = np.delete(points, [c_1, c_2], axis=0)
points = np.vstack([points, centroid])

Enfin, on recalcule la matrice des distances :

In [ ]:
distances = pdist(points) ** 2
distance_matrix = squareform(distances)

### Autres HAC

Parmi toutes les techniques de partitionnement hiérarchique ascendant, citons encore **le clustering par liens médians** qui calcule la distance médiane entre les points des clusters, ou encore **le clustering par liaisons moyennes**, qui établit la moyenne entre toutes les paires de distances dans les clusters :

![Average-linkage clustering](./figs/average-linkage_clustering.svg)

Et, pour la dernière, nous avons réservé **la méthode de Ward** qui minimise la somme des carrés des écarts au sein des clusters en fusionnant les paires de clusters qui entraînent la plus petite augmentation de la variance totale.

Pour toutes, le module `scipy.cluster.hierarchy` se charge des calculs grâce à une méthode `.linkage()` qui accepte un argument `method` :

In [ ]:
# Ward's linkage clustering
Z = hierarchy.linkage(df, method='ward')

Le module dispose également d’une méthode `.dendogram()` pour afficher le résultat du regroupement :

In [ ]:
_ = hierarchy.dendrogram(
    Z,
    labels=df.index,
    leaf_rotation=90,
    leaf_font_size=10,
    color_threshold=0.7 * max(Z[:, 2])
)

## Méthodes de partitionnement non-hiérarchiques

Si les techniques HAC reposent sur l’idée de regrouper progressivement les éléments proches dans un espace vectoriel, les méthodes de partitionnement non-hiérarchiques ne vont pas fonder de préjugés sur la ressemblance entre les données.

### L’algorithme des *k*-moyennes

Il s’agit de la méthode de partitionnement la plus courante en analyse des données. Elle consiste à délimiter *k* clusters à la double condition qu’un cluster soit constitué au moins d’une donnée et que chaque donnée appartienne à un seul cluster. Rien de nouveau dans la formulation, mais la construction des clusters n’est cette fois-ci pas aussi rigide qu’avec les techniques HAC : lorsqu’un cluster est constitué, il peut à l’étape suivante se scinder ou fusionner avec un autre.

On considère ces étapes :

1. On détermine arbitrairement un nombre *k* de centroïdes ;
2. les données sont assignées au cluster le plus proche ;
3. les centroïdes sont recalculés ;
4. il convient enfin de répéter les étapes 2 et 3 tant que nécessaire.

![*k*-means clustering](./figs/kmeans.svg)

#### Préparer les données

Les algorithmes contenus dans les bibliothèques spécialisées nécessitent presque toujours de normaliser les données avant de les injecter dans les fonctions statistiques. Pour rappel, l’opération consiste, pour chaque variable, à retirer à une observation la moyenne puis à diviser par l’écart-type, selon l’expression :

$$
Z = \frac{X − \mu}{\sigma}
$$

Avec *Numpy* et *Pandas*, on peut procéder en une seule passe si on n’omet pas le paramètre `axis=0` qui opère colonne par colonne :

In [ ]:
df_scaled = (df.values - np.mean(df.values, axis=0)) / np.std(df.values, axis=0)

#### Déterminer le nombre initial de centroïdes

Comme l’assignation d’un point à un cluster est au départ aléatoire, que le nombre de centroïdes est arbitraire et que l’algorithme ne garantit pas la meilleure classification possible, il est nécessaire d’effectuer plusieurs tests et d’évaluer leurs performances avant de figer les paramètres du modèle.

Concernant le nombre de centroïdes, plutôt que de partir à l’aveuglette, on peut le décider à partir d’un diagramme d’éboulis qui permet de visualiser l’évolution de l’inertie (somme des distances des points au centre des clusters) en fonction du nombre de clusters :

In [ ]:
# distance matrix
distances = pdist(df_scaled, metric='euclidean')
distance_matrix = squareform(distances)

# range of clusters to try
n_clusters_range = range(1, 11)

# evolution of inertia
inertia = []

# calculate inertia for each nb of clusters
for n_clusters in n_clusters_range:
    centroids, _ = kmeans(df_scaled, n_clusters)
    _, distortion = vq(df_scaled, centroids)
    inertia.append(distortion.sum())

plt.plot(n_clusters_range, inertia, marker='o', linestyle='-', markersize=8, linewidth=2)
plt.xlabel('Number of clusters', fontsize=12)
plt.ylabel('Inertia', fontsize=12)
plt.title('Elbow method for optimal number of clusters', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

#### Interpréter les clusters

La fonction `kmeans()` de *Scipy* permet ensuuite d’appliquer l’algorithme des *k*-moyennes :

In [ ]:
n_clusters = 6

centroids, _ = kmeans(df_scaled, n_clusters)
clusters, _ = vq(df_scaled, centroids)

La variable `clusters` enregistre désormais les étiquettes des différents regroupements, tant et si bien qu’il est possible de la réinjecter dans le *data frame* pour une meilleure lisibilité :

In [ ]:
df['Cluster'] = clusters

display(df)

L’un des premiers marqueurs du partitionnement serait de compter le nombre d’observations dans un cluster afin de révéler des déséquilibres dans la distribution :

In [ ]:
df.Cluster.value_counts()

La variable `centroids` permet quant à elle de visualiser les centres des clusters :

In [ ]:
print(centroids)

Chaque vecteur ligne de la matrice représente un cluster et chaque vecteur colonne une variable (Sciences, Politique, Littérature…). Les valeurs positives et négatives indiquent quelle variable est dominante pour tel ou tel cluster afin de révéler les influences. Si par exemple le cluster n°2 dispose de valeurs fortes en Philosophie et Littérature, cela signifie que ces catégories ont eu beaucoup d’importance lors du calcul du centroïde. En opérant ainsi cluster par cluster, on peut révéler les catégories qui les différencient.

Visuellement, l’analyse sera facilitée avec une carte de chaleur (*heatmap*) :

In [ ]:
# centroids into a dataframe for better visualization
columns = df.columns[:-1] # cause we've just added another column 'Cluster'
centroids_df = pd.DataFrame(centroids, columns=columns)

# heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(centroids_df, annot=True, cmap="coolwarm", cbar=True, linewidths=0.5)
plt.title("Heatmap")
plt.xlabel("Features")
plt.ylabel("Clusters")
plt.show()

Il n’est pas toujours simple de tirer des conclusions d’un partitionnement des données. Si l’on relève des déséquilibres dans la distribution des observations ou si une trop forte influence d’une caractéristique sur un cluster, cela peut simplement venir de la nature même des données. Dans les autres cas, on peut toutefois émettre des hypothèses :

- Des clusters déséquilibrés suggèrent de revoir le nombre de clusters à la hausse ;
- des clusters très petits peuvent révéler l’existence de données aberrantes qui faussent le partitionnement ;
- des caractéristiques trop fortes indiquent sans doute un biais à évaluer.

La solution n’est pas plus simple à trouver. Bien souvent, il faudra multiplier les techniques afin d’obtenir les meilleurs résultats.

## Évaluer la qualité d’une partition d’un ensemble

### Le coefficient de silhouette

Pour évaluer si un point donné a correctement été attribué à son cluster, on utilise ordinairement une mesure appelée **coefficient de silhouette**, qui se calcule grâce à l’expression :

$$
S(i) = \frac{b(i) - a(i)}{\max(a(i), b(i))}
$$

Où :

- $a(i)$ est la distance moyenne entre le point *i* et tous les autres points du même cluster ;
- $b(i)$ est la distance moyenne entre le point *i* et tous les points du cluster le plus proche (autre que le sien).

Prenons quatre points situés dans un espace bidimensionnel pour lesquels nous avons déjà effectué un partitionnement :

||x|y|cluster|
|-|-|-|:-:|
|**A**|1|2|1|
|**B**|2|3|2|
|**C**|8|9|2|
|**D**|9|8|2|

Nous souhaitons savoir si le point *B* a correctement été assigné au cluster 2.

#### 1e étape : calcul de la distance moyenne de *B* dans son propre cluster